# Stock List and Info

## Import

Libraries

In [29]:
import requests
import os
import pandas as pd
import json

Definitions

In [4]:
# current dir + data
download_dir = os.path.join(os.getcwd(), '../data')

## Get ASX List

In [5]:
# todo: does this work tmr
# The URL you provided
url = "https://asx.api.markitdigital.com/asx-research/1.0/companies/directory/file?access_token=83ff96335c2d45a094gics_df02a206a39ff4"

# Download the file
response = requests.get(url)

# Save the file if the request is successful
if response.status_code == 200:
    with open(f"{download_dir}/asx_companies_directory.csv", "wb") as file:
        file.write(response.content)
    print("File downloaded successfully!")
else:
    print(f"Failed to download file. Status code: {response.status_code}")

File downloaded successfully!


In [23]:
asx_companies = pd.read_csv("../data/asx_companies_directory.csv")

asx_companies['Market Cap'] = asx_companies['Market Cap'].apply(lambda x: int(x) if x.isnumeric() else 0)

In [30]:
ASX_codes = asx_companies["ASX code"].values

In [31]:
asx300 = asx_companies[~asx_companies['Market Cap'].isin(
    ['SUSPENDED', '---', '--'])].sort_values(['Market Cap'], ascending=False).head(300)

asx300['weight'] = asx300['Market Cap'] / asx300['Market Cap'].sum()

In [28]:
asx300_json = {ticker: weight for ticker, weight in zip(asx300['ASX code'], asx300['weight'])}
with open('../data/asx300.json', 'w') as f:
    json.dump(asx300_json, f)

,ASX code,Company name,GICs industry group,Listing date,Market Cap
379,CBA,COMMONWEALTH BANK OF AUSTRALIA.,Banks,12/09/1991,239857536909
292,BHP,BHP GROUP LIMITED,Materials,13/08/1885,198846187558
484,CSL,CSL LIMITED,"Pharmaceuticals, Biotechnology & Life Sciences",08/06/1994,143034618415
1219,NAB,NATIONAL AUSTRALIA BANK LIMITED,Banks,31/01/1962,121367520661
1878,WBC,WESTPAC BANKING CORPORATION,Banks,18/07/1970,114368517353
...,...,...,...,...,...
426,CIN,CARLTON INVESTMENTS LIMITED,Not Applic,30/06/1970,781549777
657,EQT,EQT HOLDINGS LIMITED,Financial Services,06/08/1985,778694998
1512,RFF,RURAL FUNDS GROUP,Equity Real Estate Investment Trusts (REITs),14/02/2014,777892024
1525,RIC,RIDLEY CORPORATION LIMITED,"Food, Beverage & Tobacco",13/08/1987,776948474


## Get gics sector and industry

In [11]:
import yfinance as yf

# Create a dataframe to store stock information
stock_data = []

for ticker in ASX_codes:
    ticker += ".AX"
    stock = yf.Ticker(ticker)
    info = stock.info
    stock_data.append({
        'Ticker': ticker,
        'Name': info.get('shortName', 'N/A'),
        'Market Cap': info.get('marketCap', 'N/A'),
        'Sector': info.get('sector', 'N/A'),
        'Industry': info.get('industry', 'N/A'),
    })

# Convert the data into a DataFrame
gics_df = pd.DataFrame(stock_data)

# Display the DataFrame
display(gics_df)

,Ticker,Name,Market Cap,Sector,Industry
0,14D.AX,1414DEGREE FPO [14D],16671830,Utilities,Utilities - Independent Power Producers
1,1AD.AX,ADALTA FPO [1AD],11517914,Healthcare,Biotechnology
2,1AE.AX,AURORAENER FPO [1AE],7699752,Energy,Uranium
3,1AI.AX,ALGORAE FPO [1AI],11811730,Healthcare,Biotechnology
4,1CG.AX,1CLICKLIFE FPO [1CG],7747930,Technology,Software - Application
...,...,...,...,...,...
1950,ZLD.AX,ZELIRATHER FPO [ZLD],7716096,Healthcare,Biotechnology
1951,ZMI.AX,ZINCOFIRE FPO [ZMI],2125240,Basic Materials,Other Industrial Metals & Mining
1952,ZMM.AX,ZIMILMITED FPO [ZMM],1519980,Industrials,Electrical Equipment & Parts
1953,ZNC.AX,ZENITH MIN FPO [ZNC],15152383,Basic Materials,Other Industrial Metals & Mining


In [12]:
gics_df.to_csv('../data/asx_gics.csv', index=False)